In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [2]:
df_ = pd.read_csv('LoanStats3c.csv', skiprows =[0], low_memory=False)

In [3]:
df_.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code
0,36805548,39558264,10400,10400,10400,36 months,6.99%,321.08,A,A3,...,0,0,0,Mar-2015,321.08,Apr-2015,Mar-2015,0,59,1
1,38098114,40860827,15000,15000,15000,60 months,12.39%,336.64,C,C1,...,0,0,0,Mar-2015,336.64,Apr-2015,Mar-2015,0,NaN,1
2,37822187,40585251,9600,9600,9600,36 months,13.66%,326.53,C,C3,...,0,0,0,Mar-2015,326.53,Apr-2015,Mar-2015,0,NaN,1
3,37662224,40425321,7650,7650,7650,36 months,13.66%,260.20,C,C3,...,0,0,0,Mar-2015,260.20,Apr-2015,Mar-2015,0,NaN,1
4,37612354,40375473,12800,12800,12800,60 months,17.14%,319.08,D,D4,...,0,0,0,Mar-2015,319.08,Apr-2015,Mar-2015,0,36,1


In [4]:
lending_data = df_[['annual_inc','home_ownership','int_rate']]
lending_data.head()

,annual_inc,home_ownership,int_rate
0,58000,MORTGAGE,6.99%
1,78000,RENT,12.39%
2,69000,RENT,13.66%
3,50000,RENT,13.66%
4,125000,MORTGAGE,17.14%


In [5]:
def int_rate(dt):
    dt = str(dt).strip('%')
    return float(dt)

def home_ownership(dt):
    if dt == 'MORTGAGE':
        return 0 
    elif dt == 'RENT':
        return 1 
    elif dt == 'OWN':
        return 2
    else:
        return 0

In [6]:
lending_data['int_rate'] = lending_data.int_rate.apply(int_rate)
lending_data['home_ownership'] = lending_data.home_ownership.apply(home_ownership)

/Users/ikeokonkwo/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/ikeokonkwo/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app


In [7]:
lending_data = lending_data.dropna()

data =  lending_data[['annual_inc','home_ownership']]
target = lending_data['int_rate']

lending_data.head()

,annual_inc,home_ownership,int_rate
0,58000,0,6.99
1,78000,1,12.39
2,69000,1,13.66
3,50000,1,13.66
4,125000,0,17.14


In [8]:
from sklearn.linear_model import LinearRegression 
from sklearn import metrics 

from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3)
print X_train.shape, X_test.shape

model=LinearRegression() 
model.fit(X_train, y_train) 
print(model)
# make predictions
expected = y_test 
predicted = model.predict(X_test) 


(164940, 2) (70689, 2)
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)


In [9]:
# summarize the fit of the model 
mse=np.mean((predicted-expected)**2) 
print 'Mean Squared Error :', mse 
print 'r-squared :', metrics.r2_score(expected, predicted)

Mean Squared Error : 18.6078076008
r-squared : 0.00695609272549


#### The R-square is actually quite small 

In [11]:
## Re running some of your code
df = df_
df.dropna(inplace=True)

df['annual_inc'] = df['annual_inc'].map(lambda x: float(x))

df['int_rate'] = df['int_rate'].map(lambda x:
                                    round(float(x.rstrip('%')) / 100, 4))

df['home_ownership'] = df['home_ownership'].map(lambda x:
                                                int(1 if x == 'MORTGAGE' else 1
                                                    if x == 'OWN' else 0))

intrate = df['int_rate']
home = df['home_ownership']
anninc = df['annual_inc']

y = np.matrix(intrate).transpose()

x1 = np.matrix(anninc).transpose()
x2 = np.matrix(home).transpose()

x = np.column_stack([x1])

In [12]:
X = sm.add_constant(x1)
model = sm.OLS(y, X)
f = model.fit()

print('The regression results for just annual income are:')
print(f.summary())


The regression results for just annual income are:
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.975
Date:                Sat, 18 Apr 2015   Prob (F-statistic):              0.160
Time:                        17:52:02   Log-Likelihood:                 1235.2
No. Observations:                 636   AIC:                            -2466.
Df Residuals:                     634   BIC:                            -2457.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
c

In [13]:
z = np.column_stack([x1, x2])

Z = sm.add_constant(z)
model2 = sm.OLS(y, Z)
g = model2.fit()

print('The regression results for both annual income and home ownership are:')
print(g.summary())

The regression results for both annual income and home ownership are:
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     2.461
Date:                Sat, 18 Apr 2015   Prob (F-statistic):             0.0861
Time:                        17:52:04   Log-Likelihood:                 1236.7
No. Observations:                 636   AIC:                            -2467.
Df Residuals:                     633   BIC:                            -2454.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
-------------------------------------------------------------